# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import corr

In [2]:
spark = SparkSession.builder.appName('Logistic_regression').getOrCreate()

In [3]:
df = spark.read.csv(r'F:\backup\practice\Apache_spark\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression\customer_churn.csv',header=True, inferSchema=True)

In [4]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
df_model = df.select(['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Location',
 'Company',
 'Churn'])

In [36]:
final_data = df_model.dropna(how='any')

In [37]:
final_data.count()

900

In [9]:
#Loc_ind = StringIndexer(inputCol='Location',outputCol='Loc_index')
#Loc_en = OneHotEncoder(inputCol='Loc_index',outputCol='Loc_vec')

In [10]:
#com_ind = StringIndexer(inputCol='Company',outputCol='com_index')
#com_en = OneHotEncoder(inputCol='com_index',outputCol='com_vec')

In [38]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites']
,outputCol='features')

In [12]:
lr = LogisticRegression(featuresCol='features',labelCol='Churn')

In [13]:
from pyspark.ml import Pipeline

In [39]:
pipeline = Pipeline(stages=[assembler,lr])

In [40]:
train, test = final_data.randomSplit([0.7,0.3])

In [41]:
train.describe().show()

+-------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+------------------+
|summary|              Age|    Total_Purchase|            Years|         Num_Sites|            Location|             Company|             Churn|
+-------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+------------------+
|  count|              634|               634|              634|               634|                 634|                 634|               634|
|   mean|41.71135646687697|10060.237776025233|5.332018927444795| 8.600946372239747|                null|                null| 0.167192429022082|
| stddev|6.104174397616928| 2414.746912804822| 1.23937176141395|1.7614618696215307|                null|                null|0.3734422146933869|
|    min|             26.0|             100.0|              1.0|               4.0|00154 Ashley Cour...|          Acosta PLC|     

In [42]:
test.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+---------------+-------------------+
|summary|               Age|    Total_Purchase|             Years|         Num_Sites|            Location|        Company|              Churn|
+-------+------------------+------------------+------------------+------------------+--------------------+---------------+-------------------+
|  count|               266|               266|               266|               266|                 266|            266|                266|
|   mean|42.067669172932334|10068.988270676677| 5.132857142857141| 8.556390977443609|                null|           null|0.16541353383458646|
| stddev| 6.187260787794395| 2398.561704969392|1.3464511405373936|1.7757848494300927|                null|           null|0.37225374036048403|
|    min|              22.0|            3263.0|              1.58|               3.0|00103 Jeffrey Cre...|Abbott-Thompson|                  0|

In [43]:
lr_model = pipeline.fit(train)

In [44]:
lr_model.stages

[VectorAssembler_4015b6428aece7c0f91d, LogisticRegression_4e1295828f1e39fa2d18]

In [45]:
res = lr_model.transform(test)

In [46]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [47]:
res.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



In [48]:
auc = my_eval.evaluate(res)

In [49]:
auc

0.7864455364455364

In [50]:
# Use all the data to fit the final model (Train + Test)

final_model = pipeline.fit(final_data)

In [52]:
new_df = spark.read.csv(r'F:\backup\practice\Apache_spark\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression\new_customers.csv',header=True, inferSchema=True)

In [53]:
new_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [57]:
# Apply the model on blind dataset

new_res = final_model.transform(new_df)

In [55]:
new_res.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,7.7...|[2.41832366013597...|[0.91821394450559...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,9.2...|[-5.9906678174824...|[0.00249574827398...|  

In [56]:
new_res.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       1.0|
+----------+

